In [1]:
import os
from os.path import join
import sys
import cv2 as cv
import numpy as np
from numpy.random import RandomState
import pickle
import matplotlib.pyplot as plt

import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import regularizers
from keras import backend as K
from keras.callbacks import EarlyStopping, ModelCheckpoint

c:\users\joear\appdata\local\programs\python\python36\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
# Nicely formatted time string
def hms_string(sec_elapsed):
    h = int(sec_elapsed / (60 * 60))
    m = int((sec_elapsed % (60 * 60)) / 60)
    s = sec_elapsed % 60
    return "{}:{:>02}:{:>05.2f}".format(h, m, s)

In [3]:
def pad(arr,grayscale):
    """ Pads an image if taken near the edge """
    if grayscale:
        arr = np.reshape(arr,(arr.shape[0],arr.shape[1],1))
        r = np.zeros((50,50,1))
        r[:arr.shape[0],:arr.shape[1],:arr.shape[2]] = arr
    else:
        r = np.zeros((50,50,3))
        r[:arr.shape[0],:arr.shape[1],:arr.shape[2]] = arr
    return r

In [4]:
def random_np(x_np, y_np):
    """ randomises a numpy array """
    prng = RandomState(0)
    randomise = prng.permutation(x_np.shape[0])
    return x_np[randomise], y_np[randomise]

In [5]:
def split_np(x_data, y_data, percent):
    """ splits a numpy array into testing and training """
    position = int(len(x_data) * (1-percent))
    x_train, x_test = x_data[:position], x_data[position:]
    y_train, y_test = y_data[:position], y_data[position:]
    return x_train, y_train, x_test, y_test

In [6]:
def load_data(randomise=True,grayscale=True):
    """ Loads in image data as numpy arrays """
    x_values = []
    y_values = []
    none_count = 0 
    filedir = join(os.getcwd(),"labels")
    for file in os.listdir(filedir):
        if file.endswith(".jpg"):
            if grayscale:
                img = cv.imread(join(filedir,file), cv.IMREAD_GRAYSCALE)
            else: 
                img = cv.imread(join(filedir,file), cv.IMREAD_UNCHANGED)
            if not img is None:
                img = pad(img,grayscale)
                x_values.append(img)
                ball_type = int(file.split("_")[2])
                
                if ball_type == 0:    
                    y_values.append([0,1])
                else:
                    y_values.append([1,0])    
            else:
                none_count += 1
    if none_count > 0:
        print("None count: ", none_count)
    shape = list(x_values[0].shape)
    shape[:0] = [len(x_values)]
    x_np = np.concatenate(x_values).reshape(shape)
    y_np = np.array(y_values)
    x_np, y_np = random_np(x_np, y_np)
    return x_np, y_np

In [7]:
def norm_x(x_train, x_test):
    x_train = x_train.astype('float32')
    x_test = x_test.astype('float32')
    x_train /= 255
    x_test /= 255
    return x_train, x_test

In [8]:
x_np, y_np = load_data(grayscale=False)
x_train, y_train, x_test, y_test = split_np(x_np, y_np,0.2)
x_train, x_test = norm_x(x_train, x_test)

num_classes = 2

#y_train = keras.utils.to_categorical(y_train, num_classes)
#y_test = keras.utils.to_categorical(y_test, num_classes)

print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)
print('y_train shape:', y_train.shape)
print('y_test shape:', y_test.shape)
print("Training samples: {}".format(x_train.shape[0]))
print("Test samples: {}".format(x_test.shape[0]))

None count:  43
x_train shape: (1292, 50, 50, 3)
x_test shape: (323, 50, 50, 3)
y_train shape: (1292, 2)
y_test shape: (323, 2)
Training samples: 1292
Test samples: 323


In [9]:
input_shape = x_train.shape[1:]

model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])
monitor = EarlyStopping(monitor='val_loss', min_delta=1e-3, patience=25, verbose=1, mode='auto')
checkpointer = ModelCheckpoint(filepath="dnn/tmp_best_weights_bin_cnn.hdf5", verbose=0, save_best_only=True) # save best model

In [10]:
import tensorflow as tf
import time

batch_size = 128
epochs = 1000

start_time = time.time()

model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=2,
          validation_data=(x_test, y_test),
          callbacks=[monitor,checkpointer])
model.load_weights('dnn/tmp_best_weights.hdf5') # load weights from best model


save_dir = join(os.getcwd(),"dnn")
save_path = join(save_dir,str(int(start_time)) + "_cnn.h5")
model.save(save_path)

score = model.evaluate(x_test, y_test, verbose=2)
print('Test loss: {}'.format(score[0]))
print('Test accuracy: {}'.format(score[1]))

elapsed_time = time.time() - start_time
print("Elapsed time: {}".format(hms_string(elapsed_time)))

Train on 1292 samples, validate on 323 samples
Epoch 1/1000
 - 47s - loss: 0.6303 - acc: 0.6238 - val_loss: 0.5495 - val_acc: 0.6471
Epoch 2/1000
 - 34s - loss: 0.5495 - acc: 0.6711 - val_loss: 0.3451 - val_acc: 0.6471
Epoch 3/1000
 - 24s - loss: 0.3226 - acc: 0.8181 - val_loss: 0.1034 - val_acc: 0.9907
Epoch 4/1000
 - 34s - loss: 0.1248 - acc: 0.9512 - val_loss: 0.0049 - val_acc: 1.0000
Epoch 5/1000
 - 25s - loss: 0.4006 - acc: 0.9373 - val_loss: 0.0092 - val_acc: 1.0000
Epoch 6/1000
 - 23s - loss: 0.0512 - acc: 0.9876 - val_loss: 0.0010 - val_acc: 1.0000
Epoch 7/1000
 - 24s - loss: 0.0328 - acc: 0.9915 - val_loss: 1.2480e-04 - val_acc: 1.0000
Epoch 8/1000
 - 23s - loss: 0.0228 - acc: 0.9946 - val_loss: 3.6446e-05 - val_acc: 1.0000
Epoch 9/1000
 - 23s - loss: 0.0151 - acc: 0.9961 - val_loss: 1.4572e-05 - val_acc: 1.0000
Epoch 10/1000
 - 37s - loss: 0.0197 - acc: 0.9961 - val_loss: 8.7032e-06 - val_acc: 1.0000
Epoch 11/1000
 - 37s - loss: 0.0099 - acc: 0.9977 - val_loss: 4.9889e-06 - v

In [12]:
from keras.models import load_model
import os
from os.path import join

save_dir = join(os.getcwd(),"dnn")
save_path = join(save_dir,"1528478520_cnn.h5")

model2 = load_model(save_path)
pred = model2.predict(x_test)

In [13]:
predict_classes = np.argmax(pred,axis=1)
expected_classes = np.argmax(y_test,axis=1)
print("Predictions: {}".format(predict_classes))
print("Expected: {}".format(expected_classes))

Predictions: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
Expected: [0 1 1 1 0 1 1 1 1 0 0 1 0 1 1 1 1 1 0 0 0 1 0 0 1 1 1 1 1 0 1 1 1 0 1 1 0
 1 0 1 1 1 1 1 1 1 1 1 1 0 1 1 1 0 1 1 1 0 0 1 0 0 0 1 0 1 0 1 1 0 0 1 1 0
 0 1 0 1 0 1 1 0 0 1 1 1 0 1 0 1 1 1 1 0 0 0 1 0 1 0 0 1 0 0 1 0 0 0 0 1 0
 1 1 1 1 0 0 1 0 1 1 1 0 1 0 0 1 1 0 1 0 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 0 1
 1 1 0 1 0 1 1 1 0 1 

In [14]:
from sklearn.metrics import accuracy_score
correct = accuracy_score(expected_classes,predict_classes)
print("Accuracy: {}".format(correct))

Accuracy: 0.35294117647058826
